In [1]:
import os
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [2]:
sequence_length = 2500  #序列长度，最大帧为300，但这里还需要更改
input_size = 75       #输入数据特征大小 3（x,y,z）*25（关节数量）
hidden_size = 128     #隐藏层数据特征大小,即每个时间步对应的ht的维数
num_layers = 2        #隐藏层层数
num_classes = 30      #结果类数
batch_size = 20     #一个batch大小
num_epochs = 100       #epoch数目
learning_rate = 0.001  #学习率

In [3]:
class Feeder(torch.utils.data.Dataset): 
    def __init__(self,
                 data_path,
                 label_path,
                 window_size=-1,
                 debug=False,
                 mmap=True):
        self.debug = debug
        self.data_path = data_path
        self.label_path = label_path
        self.window_size = window_size
        self.load_data(mmap)
        
    def load_data(self, mmap):
        # data: N C V T M

        self.label = np.load(label_path)
        
        self.data = np.load(data_path)

        self.N, self.C, self.T, self.V, self.M = self.data.shape

    # 获取数据集大小
    def __len__(self):
        return len(self.label)

    # 用于获取某一个数据的函数
    def __getitem__(self, index):
        data_numpy = np.array(self.data[index])
        label = self.label[index]

        return data_numpy, label

In [4]:
data_path = '/Users/kongxuwen/Desktop/竞赛/滑冰/train_dataset/train_data.npy'
label_path = '/Users/kongxuwen/Desktop/竞赛/滑冰/train_dataset/train_label.npy'

In [5]:
np.load(label_path)

array([27, 27, 27, ..., 11, 11, 11])

In [6]:
train_set = Feeder(data_path,label_path)
train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True)

In [7]:
class HRNN(nn.Module):
    # 实现三层架构，即首先经过两层普通BRNN并经过全连接层融合，最后经过一层LSTM的BRNN，然后用FC表示
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(HRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        #如果要使用反向的传递，则需令bidirectional=True
        # batch_first代表传入数据为（batch,seq,feature)的顺序 否则Pytroch所有RNN网络默认输入结构为(seq,batch,feature)
        # batch_first = true代表输入X为 batch_size,seq_len,input_size
        self.rnn = nn.RNN(int(input_size/5), int(hidden_size/4), num_layers, batch_first=True, bidirectional=True)
        self.rnn2 = nn.RNN(hidden_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.lstm = nn.LSTM(int(hidden_size/4*2*2), hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        
        #如果使用了反向传递，则需要将hidden_size*2!
        self.fc = nn.Linear(hidden_size*2*sequence_length, num_classes)
        self.fs1 = nn.Linear(hidden_size,hidden_size)
        self.fs2 = nn.Linear(hidden_size*4,hidden_size)
        
    def forward(self, x):
        # 输入：
        # X为batch_size*seq_len*input_size(batch_first=true时)
        
        # 输出：
        # 输出为out,(hn,cn)
        # out(seq_len, batch_size, num_directions*hidden_size) 即为[h1,h2,...,hseq_len]
        # 即out = torch.Size([1000, 28, 128])
        # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        '''
        in2_p1 = torch.zeros(400,28,128)
        in2_p2 = torch.zeros(400,28,128)
        in2_p3 = torch.zeros(400,28,128)
        in2_p4 = torch.zeros(400,28,128)
        
        in3_p1 = torch.zeros(800,28,128)
        in3_p2 = torch.zeros(800,28,128)
        
        in4_p1 = torch.zeros(1600,28,128)
        '''
        # layer1:即分成五个部分利用rnn进行分别建模 75/5
        # step1:将五个部分分别经过rnn层
        (x_p1,x_p2,x_p3,x_p4,x_p5) = torch.chunk(x, 5, dim = 2)
        out1_p1,_ = self.rnn(x_p1)#(20,2500,64)
        out1_p2,_ = self.rnn(x_p2)
        out1_p3,_ = self.rnn(x_p3)
        out1_p4,_ = self.rnn(x_p4)
        out1_p5,_ = self.rnn(x_p5)
        # 经过第一个RNN得到的是五个子部分的表示
        print(out1_p1.shape)
        
        # step2:利用全连接层进行特征融合
        # 先进行特征拼接
        temp2_p1 = torch.cat((out1_p1,out1_p2),2) #(20,2500,128/4*2*2),第一个2为双向乘的，第二个2为两个并在一起乘的
        temp2_p2 = torch.cat((out1_p1,out1_p3),2)
        temp2_p3 = torch.cat((out1_p1,out1_p4),2)
        temp2_p4 = torch.cat((out1_p1,out1_p5),2)
        print(temp2_p1.shape)
        # 再进行特征融合
#         seqs = temp2_p1.size(1)
        in2_p1 = F.relu(self.fs1(temp2_p1))
#         for seq in range(seqs):
#             temp = torch.squeeze(temp2_p1[:,seq,:],dim=1) #删除这个维度
#             in2_p1_i = F.relu(self.fs1(temp))
#             in2_p1.append(in2_p1_i)
#         in2_p1 = torch.stack(in2_p1,dim = 1)
        in2_p2 = F.relu(self.fs1(temp2_p2))
#         for seq in range(seqs):
#             temp = torch.squeeze(temp2_p2[:,seq,:],dim=1) #删除这个维度
#             in2_p2_i = F.relu(self.fs1(temp))
#             in2_p2.append(in2_p2_i)
#         in2_p2 = torch.stack(in2_p2,dim = 1)
        in2_p3 = F.relu(self.fs1(temp2_p3))
#         for seq in range(seqs):
#             temp = torch.squeeze(temp2_p3[:,seq,:],dim=1) #删除这个维度
#             in2_p3_i = F.relu(self.fs1(temp))
#             in2_p3.append(in2_p3_i)
#         in2_p3 = torch.stack(in2_p3,dim = 1)
        in2_p4 = F.relu(self.fs1(temp2_p4))
#         for seq in range(seqs):
#             temp = torch.squeeze(temp2_p4[:,seq,:],dim=1) #删除这个维度
#             in2_p4_i = F.relu(self.fs1(temp))
#             in2_p4.append(in2_p4_i)
#         in2_p4 = torch.stack(in2_p4,dim = 1)
        print(in2_p1.shape)
        
        # layer2:用4个部分进行输入，得到结果经过融合层变成两部分
        # step1:四个部分分别经过第二个rnn层
        out2_p1,_ = self.rnn2(in2_p1)#(20,2500,256)
        out2_p2,_ = self.rnn2(in2_p2)
        out2_p3,_ = self.rnn2(in2_p3)
        out2_p4,_ = self.rnn2(in2_p4)
        print(out2_p1.shape)
        # step2:利用全连接层进行特征融合
        temp3_p1 = torch.cat((out2_p1,out2_p2),2)#(20,2500,512)
        temp3_p2 = torch.cat((out2_p3,out2_p4),2)
        print(temp3_p1.shape)
        seqs = temp3_p1.size(1)
        in3_p1 = F.relu(self.fs2(temp3_p1))
#         for seq in range(seqs):
#             temp = torch.squeeze(temp3_p1[:,seq,:],dim=1) #删除这个维度
#             in3_p1_i = F.relu(self.fs2(temp))
#             in3_p1.append(in3_p1_i)
#         in3_p1 = torch.stack(in3_p1,dim = 1)
        
        in3_p2 = F.relu(self.fs2(temp3_p2))
#         for seq in range(seqs):
#             temp = torch.squeeze(temp3_p2[:,seq,:],dim=1) #删除这个维度
#             in3_p2_i = F.relu(self.fs2(temp))
#             in3_p2.append(in3_p2_i)
#         in3_p2 = torch.stack(in3_p2,dim = 1)
        print(in3_p1.shape)
        
        # layer3:将两个部分的结果再经过rnn层最终得到一个部分的结果
        # step1:将两个部分分别经过rnn
        out3_p1,_ = self.rnn2(in3_p1)#(,,256)
        out3_p2,_ = self.rnn2(in3_p2)
        print(out3_p1.shape)
        # step2.利用全连接层进行特征融合
        temp4_p1 = torch.cat((out3_p1,out3_p2),2)
        in4_p1 = F.relu(self.fs2(temp4_p1))
#         for seq in range(seqs):
#             temp = torch.squeeze(temp4_p1[:,seq,:],dim=1) #删除这个维度
#             in4_p1_i = F.relu(self.fs2(temp))
#             in4_p1.append(in4_p1_i)
#         in4_p1 = torch.stack(in4_p1,1)
        
        # layer3:整体作为输入经过lstm层得到输出
        out4,_ = self.lstm(in4_p1)
        #print(out4.shape) torch.Size([1000, 300, 128])
        # 代表仅取最后一个时间步的隐状态表示作为全连接层的输入(这显然是不合理的，因为有很多都没有到最后一帧)
        #out = self.fc(out4[:, -1, :])
        # 尝试一：将向量展平（但这样会存在很多0）
        out = self.fc(out4.reshape(out4.size(0),hidden_size*2*sequence_length))
        
        
        return out

In [8]:
import time

In [9]:
#Step6.模型使用以及损失函数、优化函数使用
model = HRNN(input_size, hidden_size, num_layers, num_classes)
model.train()
# 使用交叉熵损失函数作为目标函数
# 使用Adam作为优化函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    since = time.time()
    for batch_x,batch_y in train_loader:
        # 暂时只取了第一个身体
        batch_x = batch_x[:,:,:,:,0].view(-1,sequence_length,input_size)

        batch_x,batch_y = batch_x,batch_y

        optimizer.zero_grad()
        
        logit = model(batch_x)
        
        loss = criterion(logit,batch_y)
        print(loss)
        loss.backward()
        print('done')
        optimizer.step()
        
    now = time.time()
    print('[%d/%d epoch,%.0f secends] loss:%.1e'%(epoch+1,num_epochs,now-since,loss))


torch.Size([20, 2500, 64])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 256])
torch.Size([20, 2500, 512])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 256])
tensor(3.4019, grad_fn=<NllLossBackward0>)
done
torch.Size([20, 2500, 64])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 256])
torch.Size([20, 2500, 512])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 256])
tensor(15.6163, grad_fn=<NllLossBackward0>)
done
torch.Size([20, 2500, 64])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 256])
torch.Size([20, 2500, 512])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 256])
tensor(28.8647, grad_fn=<NllLossBackward0>)
done
torch.Size([20, 2500, 64])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 256])
torch.Size([20, 2500, 512])
torch.Size([20, 2500, 128])
torch.Size([20, 2500, 256])
tensor(12.6137, grad_fn=<NllLossBackward0>)
done
torch.Size([20, 2500, 64]